# ETAPA 1: Limpieza y preparación de datos

**Objetivo:** Comprender la estructura del dataset y generar una base consolidada y limpia.

**Rango temporal:** 1 mes de datos (ejemplo: marzo 2020).

**Dificultad:** Básica – operaciones individuales sobre un solo archivo CSV.

**Ítems solicitados:**
1. Cargar y visualizar los primeros 5 registros del archivo 01-22-2020.csv.
2. Mostrar el número total de filas y columnas del DataFrame.
3. Describir los tipos de datos (dtypes) y convertir las columnas necesarias (por ejemplo,
fechas).
4. Detectar y mostrar valores nulos o faltantes por columna.
5. Eliminar columnas irrelevantes (por ejemplo, códigos FIPS o coordenadas si no se usarán).
6. Estandarizar nombres de columnas (usar formato snake_case).
7. Homogeneizar nombres de países (ej. “US” → “United States”).
8. Convertir la columna Last_Update al formato YYYY-MM-DD.
9. Crear una columna active_cases = Confirmed - Deaths - Recovered.
10. Guardar el DataFrame limpio como covid_clean_enero2020.csv e indicar su tamaño en MB.


In [3]:
# Importar configuración centralizada
import sys
import os

# Agregar el directorio src al path para importar config
sys.path.insert(0, os.path.join('..', 'src'))
from config import COUNTRY_MAPPING, COLUMNS_TO_DROP, NUMERIC_COLUMNS, load_daily_reports, clean_covid_data

print("✓ Configuración centralizada importada")

✓ Configuración centralizada importada


In [4]:
import pandas as pd
import os

# Configuración: ruta a los archivos daily_reports locales
# Nota: el notebook está en notebooks/, por eso usamos '../' para subir un nivel
DATA_DIR = os.path.join('..', 'data', 'raw', 'COVID-19', 'csse_covid_19_data', 'csse_covid_19_daily_reports')

# Rango de fechas a cargar (marzo 2020, día por día)
dates = pd.date_range(start='2020-03-01', end='2020-03-31', freq='D')

# Lista temporal para acumular los DataFrames de cada día
daily_dataframes = []

# Leer cada archivo CSV diario y agregarlo a la lista
for date in dates:
    filename = date.strftime('%m-%d-%Y') + '.csv'  # Formato: MM-DD-YYYY
    filepath = os.path.join(DATA_DIR, filename)
    
    if not os.path.exists(filepath):
        print(f"⚠ Archivo no encontrado: {filename}")
        continue
    
    try:
        df_day = pd.read_csv(filepath)
        df_day['Date'] = date  # Agregar columna con la fecha del archivo
        daily_dataframes.append(df_day)
        print(f"✓ {filename}")
    except Exception as e:
        print(f"✗ Error en {filename}: {e}")

# Concatenar (apilar verticalmente) todos los DataFrames diarios en uno solo
if daily_dataframes:
    df_marzo = pd.concat(daily_dataframes, ignore_index=True, sort=False)
    print(f"\n{'='*60}")
    print(f"✓ Cargados {len(daily_dataframes)} archivos diarios")
    print(f"✓ Total de registros: {len(df_marzo):,}")
    print(f"✓ Período: {df_marzo['Date'].min().date()} → {df_marzo['Date'].max().date()}")
    print(f"{'='*60}")
else:
    df_marzo = pd.DataFrame()
    print("\n⚠ No se cargó ningún archivo.")
    print("Ejecuta: ./scripts/fetch_jhu_data.sh clone")

✓ 03-01-2020.csv
✓ 03-02-2020.csv
✓ 03-03-2020.csv
✓ 03-04-2020.csv
✓ 03-05-2020.csv
✓ 03-06-2020.csv
✓ 03-07-2020.csv
✓ 03-08-2020.csv
✓ 03-09-2020.csv
✓ 03-10-2020.csv
✓ 03-11-2020.csv
✓ 03-12-2020.csv
✓ 03-13-2020.csv
✓ 03-14-2020.csv
✓ 03-15-2020.csv
✓ 03-16-2020.csv
✓ 03-17-2020.csv
✓ 03-18-2020.csv
✓ 03-19-2020.csv
✓ 03-20-2020.csv
✓ 03-21-2020.csv
✓ 03-22-2020.csv
✓ 03-23-2020.csv
✓ 03-24-2020.csv
✓ 03-25-2020.csv
✓ 03-26-2020.csv
✓ 03-27-2020.csv
✓ 03-28-2020.csv
✓ 03-29-2020.csv
✓ 03-30-2020.csv
✓ 03-31-2020.csv

✓ Cargados 31 archivos diarios
✓ Total de registros: 39,198
✓ Período: 2020-03-01 → 2020-03-31


In [5]:
# 1. Cargar y visualizar los primeros 5 registros del archivo
df_marzo.head(5)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Date,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Active,Combined_Key
0,Hubei,Mainland China,2020-03-01T10:13:19,66907,2761,31536,30.9756,112.2707,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,South Korea,2020-03-01T23:43:03,3736,17,30,36.0000,128.0000,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Italy,2020-03-01T23:23:02,1694,34,83,43.0000,12.0000,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Guangdong,Mainland China,2020-03-01T14:13:18,1349,7,1016,23.3417,113.4244,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Henan,Mainland China,2020-03-01T14:13:18,1272,22,1198,33.8820,113.6140,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#2. Mostrar el número total de filas y columnas del DataFrame.
print(f"   Total de filas: {df_marzo.shape[0]:,}")
print(f"   Total de columnas: {df_marzo.shape[1]}")

   Total de filas: 39,198
   Total de columnas: 18


In [7]:
# 3. Describir los tipos de datos (dtypes) y convertir las columnas necesarias (por ejemplo,fechas)
df_marzo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39198 entries, 0 to 39197
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province/State  2993 non-null   object        
 1   Country/Region  5555 non-null   object        
 2   Last Update     5555 non-null   object        
 3   Confirmed       39198 non-null  int64         
 4   Deaths          39198 non-null  int64         
 5   Recovered       39198 non-null  int64         
 6   Latitude        5492 non-null   float64       
 7   Longitude       5492 non-null   float64       
 8   Date            39198 non-null  datetime64[ns]
 9   FIPS            30506 non-null  float64       
 10  Admin2          30712 non-null  object        
 11  Province_State  31866 non-null  object        
 12  Country_Region  33643 non-null  object        
 13  Last_Update     33643 non-null  object        
 14  Lat             33598 non-null  float64       
 15  Lo

In [8]:
#convertir las columnas necesarias (por ejemplo,fechas).
if 'Last_Update' in df_marzo.columns:
    df_marzo['Last_Update'] = pd.to_datetime(df_marzo['Last_Update'], format='mixed', errors='coerce')
elif 'Last Update' in df_marzo.columns:
    df_marzo['Last Update'] = pd.to_datetime(df_marzo['Last Update'], format='mixed', errors='coerce')

numeric_columns = ['Confirmed', 'Deaths', 'Recovered']
for col in numeric_columns:
    if col in df_marzo.columns:
        df_marzo[col] = pd.to_numeric(df_marzo[col], errors='coerce').fillna(0).astype(int)

df_marzo.dtypes

Province/State            object
Country/Region            object
Last Update               object
Confirmed                  int64
Deaths                     int64
Recovered                  int64
Latitude                 float64
Longitude                float64
Date              datetime64[ns]
FIPS                     float64
Admin2                    object
Province_State            object
Country_Region            object
Last_Update       datetime64[ns]
Lat                      float64
Long_                    float64
Active                   float64
Combined_Key              object
dtype: object

In [9]:
#4. Detectar y mostrar valores nulos o faltantes por columna
df_marzo.isnull().sum()

Province/State    36205
Country/Region    33643
Last Update       33643
Confirmed             0
Deaths                0
Recovered             0
Latitude          33706
Longitude         33706
Date                  0
FIPS               8692
Admin2             8486
Province_State     7332
Country_Region     5555
Last_Update        5555
Lat                5600
Long_              5600
Active             5555
Combined_Key       5555
dtype: int64

In [10]:
#5. Eliminar columnas irrelevantes por ejemplo, códigos FIPS o coordenadas si no se usarán).
columns_to_drop = ['FIPS', 'Admin2', 'Lat', 'Long_', 'Latitude', 'Longitude', 'Combined_Key']
df_marzo = df_marzo.drop(columns=[col for col in columns_to_drop if col in df_marzo.columns])
df_marzo.columns

Index(['Province/State', 'Country/Region', 'Last Update', 'Confirmed',
       'Deaths', 'Recovered', 'Date', 'Province_State', 'Country_Region',
       'Last_Update', 'Active'],
      dtype='object')

In [11]:
#6. Estandarizar nombres de columnas (usar formato snake_case)
df_marzo.columns = df_marzo.columns.str.lower().str.replace(' ', '_').str.replace('/', '_').str.replace('-', '_')
df_marzo.columns

Index(['province_state', 'country_region', 'last_update', 'confirmed',
       'deaths', 'recovered', 'date', 'province_state', 'country_region',
       'last_update', 'active'],
      dtype='object')

In [12]:
#6.5. Eliminar columnas duplicadas (consolidando valores)
# Los archivos de marzo 2020 tienen diferentes esquemas que generan columnas duplicadas
duplicated_cols = df_marzo.columns[df_marzo.columns.duplicated()].unique()

if len(duplicated_cols) > 0:
    print(f"⚠ Encontradas columnas duplicadas: {duplicated_cols.tolist()}")
    
    for col_name in duplicated_cols:
        # Obtener todas las columnas con este nombre
        matching_cols = [i for i, c in enumerate(df_marzo.columns) if c == col_name]
        
        # Consolidar: tomar el primer valor no nulo de cada fila
        consolidated = df_marzo.iloc[:, matching_cols[0]]
        for col_idx in matching_cols[1:]:
            consolidated = consolidated.fillna(df_marzo.iloc[:, col_idx])
        
        # Eliminar todas las columnas duplicadas
        df_marzo = df_marzo.drop(df_marzo.columns[matching_cols], axis=1)
        
        # Agregar la columna consolidada
        df_marzo[col_name] = consolidated
        print(f"  ✓ '{col_name}' consolidada")
    
    print(f"\n✓ Columnas finales: {df_marzo.columns.tolist()}")
else:
    print("✓ No hay columnas duplicadas")

⚠ Encontradas columnas duplicadas: ['province_state', 'country_region', 'last_update']
  ✓ 'province_state' consolidada
  ✓ 'country_region' consolidada
  ✓ 'last_update' consolidada

✓ Columnas finales: ['confirmed', 'deaths', 'recovered', 'date', 'active', 'province_state', 'country_region', 'last_update']
  ✓ 'last_update' consolidada

✓ Columnas finales: ['confirmed', 'deaths', 'recovered', 'date', 'active', 'province_state', 'country_region', 'last_update']


In [13]:
#7. Homogeneizar nombres de países  (ej. "US" → "United States").
# Usar el mapeo centralizado de src/config.py
country_mapping = COUNTRY_MAPPING

# Buscar todas las columnas relacionadas con país
country_cols = [col for col in df_marzo.columns if 'country' in col.lower()]

if country_cols:
    # Si ya existe 'country_region', usarla directamente
    if 'country_region' in df_marzo.columns:
        country_col = 'country_region'
    else:
        # Si hay múltiples columnas de país, consolidar en una sola
        if len(country_cols) > 1:
            print(f"⚠ Se encontraron {len(country_cols)} columnas de país: {country_cols}")
            # Tomar la primera columna no nula de cada fila
            df_marzo['country_region'] = df_marzo[country_cols[0]].fillna('')
            for col in country_cols[1:]:
                df_marzo['country_region'] = df_marzo['country_region'].where(
                    df_marzo['country_region'] != '', 
                    df_marzo[col].fillna('')
                )
            # Eliminar las columnas originales duplicadas
            df_marzo = df_marzo.drop(columns=country_cols)
            print("✓ Columnas consolidadas en 'country_region'")
        else:
            # Solo hay una columna, renombrarla
            df_marzo = df_marzo.rename(columns={country_cols[0]: 'country_region'})
        
        country_col = 'country_region'
    
    # Aplicar el mapeo de países
    df_marzo[country_col] = df_marzo[country_col].replace(country_mapping)
    print(f"✓ Top 10 países:")
    print(df_marzo[country_col].value_counts().head(10))
else:
    print("⚠ No se encontró columna de país en el DataFrame.")
    print(f"Columnas disponibles: {df_marzo.columns.tolist()}")

✓ Top 10 países:
country_region
United States     31975
China              1038
Malaysia            527
United Kingdom      367
Canada              297
Australia           255
France              180
New Zealand          93
Netherlands          73
Denmark              65
Name: count, dtype: int64


In [14]:
#8. Convertir la columna Last_Update al formato YYYY-MM-DD
if 'last_update' in df_marzo.columns:
    df_marzo['last_update'] = pd.to_datetime(df_marzo['last_update'], format='mixed', errors='coerce').dt.strftime('%Y-%m-%d')

df_marzo[['last_update']].head()

,last_update
0,2020-03-01
1,2020-03-01
2,2020-03-01
3,2020-03-01
4,2020-03-01


In [15]:
#9. Crear una columna active_cases = Confirmed - Deaths - Recovered
df_marzo['active_cases'] = df_marzo['confirmed'] - df_marzo['deaths'] - df_marzo['recovered']
df_marzo[['confirmed', 'deaths', 'recovered', 'active_cases']].head()

,confirmed,deaths,recovered,active_cases
0,66907,2761,31536,32610
1,3736,17,30,3689
2,1694,34,83,1577
3,1349,7,1016,326
4,1272,22,1198,52


In [16]:
#10. Guardar el DataFrame limpio como covid_clean_enero2020.csv e indicar su tamaño en MB.
import os

output_path = '../data/processed/covid_clean_marzo2020.csv'
df_marzo.to_csv(output_path, index=False)

file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"Archivo guardado: {output_path}")
print(f"Tamaño del archivo: {file_size_mb:.2f} MB")

Archivo guardado: ../data/processed/covid_clean_marzo2020.csv
Tamaño del archivo: 2.10 MB
